In [ ]:
### Calculating 'responsible' box 
iou_b1 = intersection_over_union(predictions[..,21:25], target[...,21:25])
iou_b2 = intersection_over_union(predictions[..,26:30], target[...,21:25])

## 21:25 has coords for box1, 26:30 for box2. The target is only one box
# 

best_box = argmax(concatenate(newaxis(iou_b1,0),newaxis(iou_b2, 0)), axis = 0) #pseudo code, We stack two (S,S) matrix : output (2,S,S). Argmax gives (1,S,S) if 1: take box2, if 0 take box1

GT labels have xmid, ymid normalized by image shape. So the same value can be used for an S*S image (that's our grid)
So cell where mid point lies
i, j = int(S*xmid), int(S*ymid)

target [i,j,20] = 1 (if zero earlier)

#note these are w.r.t. grid cell, hence should be converted as shown below 
### TODO (confirm)
x = S*x - i #gives offset top of cell
y = S*y - j 
w = w*S #cell is 1/Sth of the image, we need width w.r.t. cell size. so multiply by S   
h = h*S 

target [i,j,21:25] = [x,y,w,h] 
target [i,j, class_label] = 1 # e.g. 0:20 index represent the classes in order. So put 1 at index of relevant class 

exists_box = target[..., 20]  # Iobj_i# check whether that particular cell has an object or not. Box_exists will be a S*S matrix. 




exists_box is ${1\!\!1} i $, object exists in that cell.
exists_box*best_box is ${1\!\!1} ij $, object exists and $j^{th}$ box is responsible for predicting it (higher IOU with target). 

In [ ]:
##################################
 ### Box Coordinates ###
##################################

# we only take coordinates component if that cell has object mid point, i.e. I_obj_i (exists box)
predictions_xy_box_1 = exists_box*(
                    best_box*predictions[...,26:30])  #IOU more for box2, best_box = 1

predictions_xy_box_2 = exists_box*(
                    (1-best_box)*predictions[...,26:30])  #IOU more for box1, best_box = 0             
box_predictions =  predictions_xy_box_1 + predictions_xy_box_2  # Size: [,S,S,4]

target_xy = exists_box*target[..., 21:25] 

# We take [...,index] because we have N,S,S which we want to keep as is


#box_predictions has x,y,w,h. We need mse for x,y 
# for w,h we need to take root first 

box_predictions[..., 2:4] = torch.sign(box_predictions[..., 2:4])*(torch.sqrt(abs(box_predictions[..., 2:4] +1e-6)))  #We want to maintain the sign for gradient's sake

box_targets[..., 2:4] = torch.sqrt(box_targets[..., 2:4]) 

# we need to flatten (N, S, S, 4) into (N*S*S, 4), so until end_dim = -2
box_loss = torch.mse(torch.flatten(box_predictions, end_dim = -2),
                    torch.flatten(box_targets, end_dim = -2) )


##################################
 ### Object Loss ###
##################################

C_pred_box = best_box*predictions[..., 25:26]  + (1-best_box)*predictions[..., 20:21] # 20:21 to make sure dimension is N,S,S,1

# flatten N,S,S,1 to N*S*S,1
object_loss = torch.mse(torch.flatten(exist_box*C_pred_box),
                    torch.flatten(exist_box*targets[..., 20:21]) ) # this loss component only if object exists

##################################
 ### No Object Loss ###
##################################

# If the cell has no object, both boxes should be penalized if they detected any object. 

no_object_loss_box_1 = torch.mse(
                        torch.flatten(predictions[..., 20:21]),
                        torch.flatten(targets[..., 20:21]) )

no_object_loss_box_2 = torch.mse(
                        torch.flatten(predictions[..., 25:26]),
                        torch.flatten(targets[..., 20:21]) )  # we have only one box in target

no_object_loss = no_object_loss_box_1 + no_object_loss_box_2   



##################################
 ### Class Loss ###
##################################

torch.mse(
        torch.flatten(predictions [...,0:20], axis_end = -2))
        torch.flatten(targets [...,0:20], axis_end = -2))
        )


loss = lambda_coord*exist_box*box_loss 
       + exist_box*obj_loss 
       + lambda_no_obj*(1-exist_box)*no_obj_loss 
       + exist_box*class_loss

Note: 


We need to do NMS to find the best box out of the S*S predictions we may get for inference. For training, our predictions and labels are N,S,S,30 matrix. So no issues.  